In [29]:
import pandas as pd
import numpy as np
train_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_train.csv")
train_tag=pd.read_csv(r"D:\DA_competition\DC\data\tag_train.csv")
train_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_train.csv")

#表连接
train_op=pd.merge(train_op,train_tag,on="UID",how="left")
train_op=train_op.drop_duplicates()
train_tr=pd.merge(train_tr,train_tag,on="UID",how="left")
train_tr=train_tr.drop_duplicates()

test_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_test.csv")
test_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_test.csv")
test_op=test_op.drop_duplicates()
test_tr=test_tr.drop_duplicates()

#处理时间
train_op["hour"]=train_op.time.apply(lambda x:x.split(":")[0])
train_tr["hour"]=train_tr.time.apply(lambda x:x.split(":")[0])
test_op["hour"]=test_op.time.apply(lambda x:x.split(":")[0])
test_tr["hour"]=test_tr.time.apply(lambda x:x.split(":")[0])

D:\developsoft\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
train_fts=pd.DataFrame(pd.concat([train_op.UID,train_tr.UID],axis=0).unique())
train_fts.columns=["UID"]
train_fts.sort_values("UID",inplace=True)
test_fts=pd.read_csv(r"D:\DA_competition\DC\data\sub_sample.csv")
test_fts=test_fts.drop("Tag",axis=1)
test_fts
#31198

,UID
0,30000
1,30001
2,30002
3,30003
4,30004
5,30005
6,30006
7,30007
8,30008
9,30009


In [39]:
#operation表
#1、UID对应mode,success,os,version:cross_type
#transaction表
#1、UID对应channel、amt_src1、trans_type1、amt_src2、trans_type2、market_type：cross_type
op_fields1=["mode","success","os","version"]
tr_fields1=["channel","amt_src1","trans_type1","amt_src2","trans_type2","market_type"]
def get_discrete_fts1(data,fields):
    data_one_hot=pd.get_dummies(data[fields].applymap(str),dummy_na=True)
    data_one_hot["UID"]=data["UID"]
    fts_cross_type=data_one_hot.groupby("UID").sum().reset_index(drop=False)
    print("=====OK======")
    return fts_cross_type

In [32]:
#operation表
# 2.1、UID对应day、time、mode、success、os、version、device1、device2、geo_code种类数
# 2.2、UID对应device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub种类数
#transaction表
# 2.1、UID对应channel,day,time,amt_src1,amt_src2,merchant,code1,code2,trans_type1,trans_type2,device1,device2,geo_code,market_code,market_type种类数
# 2.2、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1数
op_fields2=["day","hour","mode","success","os","version","device1","device2",
           "geo_code","device_code1","device_code2","device_code3","mac1",
           "mac2","wifi","ip1","ip2","ip1_sub","ip2_sub"]
tr_fields2=["channel","day","hour","amt_src1","amt_src2","merchant","code1",
           "code2","trans_type1","trans_type2","device1","device2","geo_code","market_code",
           "market_type","ip1","ip1_sub","acc_id1","acc_id2","acc_id3","device_code1",
           "device_code2","device_code3","mac1"]
def get_kinds_fts2(data,fields):
    fts=data.groupby("UID")[fields].apply(lambda x:x.apply(lambda y:len(y.unique()))).reset_index(drop=False)
    print("=====OK======")
    return fts

In [33]:
#operation表
# 3.1、UID的device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub对应的用户数量均值、最大值、最小值（因为一个UID可能对应多个连续字段）
# 3.2、UID的device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub对应的用户羊毛党概率占比均值、最大值、最小值
#transaction表
# 3.1、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1对应的用户数量均值、最大值、最小值（因为一个UID可能对应多个连续字段）
# 3.2、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1对应的用户羊毛党概率占比均值、最大值、最小值
op_fields3=["device_code1","device_code2","device_code3","mac1","mac2","wifi","ip1","ip2","ip1_sub","ip2_sub"]
tr_fields3=["ip1","ip1_sub","acc_id1","acc_id2","acc_id3","device_code1","device_code2","device_code3","mac1"]
# def get_fields_as_key_fts(data,fields):
#     fts_all=pd.DataFrame(data["UID"].unique())
#     fts_all.columns=["UID"]
#     for field in fields:
        
#         d=data[["UID",field,"Tag"]].drop_duplicates().fillna(0).groupby(field)
#         f_count=d.Tag.count()
#         count_1=d.Tag.apply(lambda x:len(x[x==1]))
        
#         df=pd.merge(pd.DataFrame(f_count),pd.DataFrame(count_1),how="left",left_index=True,right_index=True)
#         df.columns=["user_counts","black_counts"]
#         df=df.reset_index(drop=False)
#         df["black_ratio"]=df.black_counts/df.user_counts
        
#         fts_mid=data[["UID",field]].fillna(0)
#         fts_mid=pd.merge(fts_mid,df,how="left",on=field).drop_duplicates()
        
#         fm_group=fts_mid.groupby("UID")
        
#         fts=pd.DataFrame(fm_group.user_counts.mean())
#         fts.columns=[field+"count_mean"]
#         fts[field+"count_max"]=fm_group.user_counts.max()
#         fts[field+"count_min"]=fm_group.user_counts.min()
#         fts[field+"ratio_mean"]=fm_group.black_counts.sum()/fm_group.user_counts.sum()
#         fts[field+"ratio_max"]=fm_group.black_ratio.max()
#         fts[field+"ratio_min"]=fm_group.black_ratio.min()
#         fts=fts.reset_index(drop=False)
#         print(fts)
#         fts_all=pd.merge(fts_all,fts,how="left",on="UID")
#     return fts_all
def get_fields_as_key_fts3(data,fields):
    fts_all=pd.DataFrame(data["UID"].unique())
    fts_all.columns=["UID"]
    for field in fields:
        
        f_count=data[["UID",field]].drop_duplicates().fillna(0).groupby(field).count()
        df=pd.DataFrame(f_count)
        df.columns=["user_counts"]
        df=df.reset_index(drop=False)
        
        fts_mid=data[["UID",field]].fillna(0)
        fts_mid=pd.merge(fts_mid,df,how="left",on=field).drop_duplicates()
        
        fm_group=fts_mid.groupby("UID")
        
        fts=pd.DataFrame(fm_group.user_counts.mean())
        fts.columns=[field+"count_mean"]
        fts[field+"count_max"]=fm_group.user_counts.max()
        fts[field+"count_min"]=fm_group.user_counts.min()
        fts=fts.reset_index(drop=False)
        fts_all=pd.merge(fts_all,fts,how="left",on="UID")
    print("=====OK======")
    return fts_all
        

In [34]:
# operation表
# transaction表
# 4、UID对应样本数
def get_sample_counts_fts4(data):
    fts=pd.DataFrame(data["UID"].value_counts().sort_index()).reset_index(drop=False)
    fts.columns=["UID","UID_sample_counts"]
    print("=====OK======")
    return fts
    

In [35]:
# transaction表
# UID对应trans_amt、bal的和、平均值、最大值、最小值
tr_fields5=["trans_amt","bal"]
def get_tr_money_fts5(data,fields):
    group=data.groupby("UID")[fields]
    fts1=group.apply(lambda x:x.sum())
    fts1.columns=[f+"_sum" for f in fields]
    fts2=group.apply(lambda x:x.max())
    fts2.columns=[f+"_max" for f in fields]
    fts3=group.apply(lambda x:x.min())
    fts3.columns=[f+"_min" for f in fields]
    fts4=group.apply(lambda x:x.mean())
    fts4.columns=[f+"_mean" for f in fields]
    fts=pd.concat([fts1,fts2,fts3,fts4],axis=1)
    fts["UID"]=fts1.index
    print("=====OK======")
    return fts

In [36]:
#transaction表
# 6、divice_code1、2，device1、mac1、geo_code缺失值，羊毛党明显偏高羊毛党：空值数量或者是否包含空值
# ip1,ip1_sub,device2也有20%偏高
#获得空值率
tr_fields6=["device_code1","device_code2","device1","mac1","geo_code","ip1","ip1_sub","device2"]
def get_tr_null_fts6(data,fields):
    group=data.groupby("UID")
    fts=pd.DataFrame(group.apply(lambda x:len(x)))
    fts.columns=["count"]
    for field in fields:
        fts=pd.concat([fts,group[field].apply(lambda x:len(x[x.isnull()]))],axis=1)
    fts=fts.div(fts["count"],axis=0)
    fts=fts.reset_index(drop=False)
    fts=fts.drop(["count"],axis=1)
    print("=====OK======")
    return fts

In [40]:
train_op_fts1=get_discrete_fts1(train_op,op_fields1)
train_tr_fts1=get_discrete_fts1(train_tr,tr_fields1)
train_op_fts2=get_kinds_fts2(train_op,op_fields2)
train_tr_fts2=get_kinds_fts2(train_tr,tr_fields2)
train_op_fts3=get_fields_as_key_fts3(train_op,op_fields3)
train_tr_fts3=get_fields_as_key_fts3(train_tr,tr_fields3)
train_op_fts4=get_sample_counts_fts4(train_op)
train_tr_fts4=get_sample_counts_fts4(train_tr)
train_tr_fts5=get_tr_money_fts5(train_tr,tr_fields5)
train_tr_fts6=get_tr_null_fts6(train_tr,tr_fields6)

=====OK======
=====OK======


In [41]:
train_fts=train_fts.merge(train_op_fts1,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts1,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts2,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts2,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts3,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts3,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts4,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts4,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts5,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts6,how="left",on="UID")

In [42]:
test_op_fts1=get_discrete_fts1(test_op,op_fields1)
test_tr_fts1=get_discrete_fts1(test_tr,tr_fields1)
test_op_fts2=get_kinds_fts2(test_op,op_fields2)
test_tr_fts2=get_kinds_fts2(test_tr,tr_fields2)
test_op_fts3=get_fields_as_key_fts3(test_op,op_fields3)
test_tr_fts3=get_fields_as_key_fts3(test_tr,tr_fields3)
test_op_fts4=get_sample_counts_fts4(test_op)
test_tr_fts4=get_sample_counts_fts4(test_tr)
test_tr_fts5=get_tr_money_fts5(test_tr,tr_fields5)
test_tr_fts6=get_tr_null_fts6(test_tr,tr_fields6)

=====OK======
=====OK======
=====OK======
=====OK======
         UID  device_code1count_mean  device_code1count_max  \
0      30000                 19357.0                  19357   
1      30001                  9679.0                  19357   
2      30002                     1.0                      1   
3      30003                     1.0                      1   
4      30005                     1.0                      1   
5      30007                     1.0                      1   
6      30008                  6453.0                  19357   
7      30009                  9679.0                  19357   
8      30010                     1.0                      1   
9      30011                     1.0                      1   
10     30013                  9679.0                  19357   
11     30014                  9679.0                  19357   
12     30015                  9679.0                  19357   
13     30017                  9679.0                  19357   

         UID  device_code3count_mean  device_code3count_max  \
0      30000                1.000000                      1   
1      30001            25829.000000                  25829   
2      30002            25829.000000                  25829   
3      30003            25829.000000                  25829   
4      30005            25829.000000                  25829   
5      30007            25829.000000                  25829   
6      30008            25829.000000                  25829   
7      30009            25829.000000                  25829   
8      30010            25829.000000                  25829   
9      30011            25829.000000                  25829   
10     30013            25829.000000                  25829   
11     30014            25829.000000                  25829   
12     30015            25829.000000                  25829   
13     30017            25829.000000                  25829   
14     30018            25829.000000                  2

         UID  mac2count_mean  mac2count_max  mac2count_min
0      30000    25575.000000          25575          25575
1      30001     8525.666667          25575              1
2      30002    12788.000000          25575              1
3      30003    17932.000000          25575          10289
4      30005    12788.000000          25575              1
5      30007    12753.333333          25575           2396
6      30008     6394.500000          25575              1
7      30009    12788.000000          25575              1
8      30010    17932.000000          25575          10289
9      30011    10289.000000          10289          10289
10     30013    12788.000000          25575              1
11     30014    12788.000000          25575              1
12     30015    12788.000000          25575              1
13     30017    12788.000000          25575              1
14     30018    10289.000000          10289          10289
15     30019    17932.000000          25575          102

         UID  ip2count_mean  ip2count_max  ip2count_min
0      30000   27887.000000         27887         27887
1      30001   27887.000000         27887         27887
2      30002   27887.000000         27887         27887
3      30003   27887.000000         27887         27887
4      30005   27887.000000         27887         27887
5      30007   27887.000000         27887         27887
6      30008    2147.000000         27887             2
7      30009   27887.000000         27887         27887
8      30010   27887.000000         27887         27887
9      30011   27887.000000         27887         27887
10     30013   13944.000000         27887             1
11     30014   27887.000000         27887         27887
12     30015   27887.000000         27887         27887
13     30017   13944.000000         27887             1
14     30018   27887.000000         27887         27887
15     30019   27887.000000         27887         27887
16     30020   13945.000000         27887       

         UID  ip1count_mean  ip1count_max  ip1count_min
0      30000       1.000000             1             1
1      30001    5536.000000         11071             1
2      30002       1.000000             1             1
3      30003       1.000000             1             1
4      30004   11071.000000         11071         11071
5      30005       1.500000             2             1
6      30006   11071.000000         11071         11071
7      30007    5536.000000         11071             1
8      30008    2215.800000         11071             2
9      30009    3899.000000         11071           307
10     30010    5536.000000         11071             1
11     30011   11071.000000         11071         11071
12     30012   11071.000000         11071         11071
13     30013       1.000000             1             1
14     30014       1.000000             1             1
15     30015       1.000000             1             1
16     30016   11071.000000         11071       

         UID  acc_id2count_mean  acc_id2count_max  acc_id2count_min
0      30000           29980.00             29980             29980
1      30001           29980.00             29980             29980
2      30002           29980.00             29980             29980
3      30003           29980.00             29980             29980
4      30004           29980.00             29980             29980
5      30005           14990.50             29980                 1
6      30006           29980.00             29980             29980
7      30007           29980.00             29980             29980
8      30008            7495.75             29980                 1
9      30009           29980.00             29980             29980
10     30010           29980.00             29980             29980
11     30011           29980.00             29980             29980
12     30012           29980.00             29980             29980
13     30013           29980.00             2998

         UID  device_code1count_mean  device_code1count_max  \
0      30000                 14866.0                  14866   
1      30001                  7433.5                  14866   
2      30002                     1.0                      1   
3      30003                     1.0                      1   
4      30004                 14866.0                  14866   
5      30005                     1.0                      1   
6      30006                 14866.0                  14866   
7      30007                  7433.5                  14866   
8      30008                  4956.0                  14866   
9      30009                 14866.0                  14866   
10     30010                  7433.5                  14866   
11     30011                 14866.0                  14866   
12     30012                 14866.0                  14866   
13     30013                     1.0                      1   
14     30014                     1.0                   

         UID  mac1count_mean  mac1count_max  mac1count_min
0      30000    14871.000000          14871          14871
1      30001     7436.000000          14871              1
2      30002        1.000000              1              1
3      30003     8012.000000           8012           8012
4      30004    14871.000000          14871          14871
5      30005        1.000000              1              1
6      30006    14871.000000          14871          14871
7      30007    11441.500000          14871           8012
8      30008     4957.666667          14871              1
9      30009    14871.000000          14871          14871
10     30010    11441.500000          14871           8012
11     30011    14871.000000          14871          14871
12     30012    14871.000000          14871          14871
13     30013        1.000000              1              1
14     30014     4006.500000           8012              1
15     30015        1.000000              1             

In [43]:
test_fts=test_fts.merge(test_op_fts1,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts1,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts2,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts2,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts3,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts3,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts4,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts4,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts5,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts6,how="left",on="UID")

In [49]:
train_fts.fillna(0,inplace=True)
test_fts.fillna(0,inplace=True)
train_fts=pd.merge(train_fts,train_tag,how="left",on="UID")
train_fts.columns

Index(['UID', 'mode_00094ae2a1d62504', 'mode_072eee5c88d380df',
       'mode_08017d2cb28c2348', 'mode_09080b31b40d57e8',
       'mode_0e72a7851fadf00b', 'mode_12845a3fe90eb1de',
       'mode_148dc2618a3a92a4', 'mode_1c341176507fbd9b',
       'mode_1cbe3d31395984d7',
       ...
       'bal_mean', 'device_code1', 'device_code2', 'device1', 'mac1',
       'geo_code', 'ip1', 'ip1_sub', 'device2', 'Tag'],
      dtype='object', length=440)

In [47]:
from sklearn.model_selection import train_test_split

In [51]:
X=np.array(train_fts.iloc[:,1:-1])
y=np.array(train_fts.iloc[:,-1])
X_train,X_verify,y_train,y_verify=train_test_split(X,y,test_size=0.2,random_state=0)
X_test=np.array(test_fts)

In [52]:
import lightgbm as lgb
train_set=lgb.Dataset(X_train,y_train)
params={'boosting':'gbdt',
        'objective': 'binary',
        'learning_rate': 0.1,
        'max_depth': 6,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'min_child_weight': 25,
        }
n_rounds=1000
clf=lgb.train(params,train_set,n_rounds)

In [54]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3


In [55]:
#验证集
y_verify_pre=clf.predict(X_verify)
tpr_weight_funtion(y_verify,y_verify_pre)

0.75362149532710276

In [56]:

train_set_for_test=lgb.Dataset(X,y)
clf_test=lgb.train(params,train_set_for_test,n_rounds)


AttributeError: module 'lightgbm' has no attribute 'predict'

In [57]:
y_submit=clf_test.predict(X_test)

In [59]:
result=pd.concat([test_fts["UID"],pd.Series(y_submit)],axis=1)

In [61]:
result.columns=["UID","Tag"]
result.to_csv(r"D:\DA_competition\DC\data\result.csv",index=False)

In [65]:
train_fts.to_csv(r"D:\DA_competition\DC\data\train_fts.csv",index=False)
test_fts.to_csv(r"D:\DA_competition\DC\data\test_fts.csv",index=False)